In [1]:
import numpy as np
np.random.seed(1)
from nltk.corpus import stopwords
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from nltk.stem.porter import PorterStemmer
from itertools import chain
from keras.preprocessing.text import one_hot
import string
import re

Using TensorFlow backend.


In [2]:
docs = ['King is man',
        'a king marries queen',
        'a queen marries king',
        'some unrelated works',
        'queen is woman']



In [3]:
#counting number of different words
# docs_temp = [row.lower() for row in docs ]
docs_temp = [elem.split() for elem in docs]
len(set(chain(*docs_temp)))

11

In [4]:
vocab_size = 20 #the nth most frequent

In [5]:
docs

['King is man',
 'a king marries queen',
 'a queen marries king',
 'some unrelated works',
 'queen is woman']

In [6]:
#using a panda dataset
import pandas as pd 
docs_panda = pd.DataFrame()
docs_panda['text'] = docs
docs_panda

,text
0,King is man
1,a king marries queen
2,a queen marries king
3,some unrelated works
4,queen is woman


# Perform these operations if we have normal raw text !

In [8]:
# - making any word starts with lowercase 
docs_panda['text'] = docs_panda['text'].apply(lambda word: word.lower())
# - removing stopwords
stop = stopwords.words('english')
docs_panda['text'] = docs_panda['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#- removing punctuation 
docs_panda['text'] = docs_panda['text'].apply(lambda row: row.translate(str.maketrans('','',string.punctuation)))
# - removing bad and useless chars (numbers @ and so on)
docs_panda['text'] = docs_panda['text'].apply(lambda row: re.sub('[/(){}\[\]\|@,;]', '', row))
docs_panda['text'] = docs_panda['text'].apply(lambda row: re.sub('[^0-9a-z #+_]', '', row))
docs_panda['text'] = docs_panda['text'].apply(lambda row: re.sub(r'\d+', '', row))

# performing stemming (#careful with this because king disappear)
# porter_stemmer = PorterStemmer()
# docs_panda['text'] = docs_panda['text'].apply(lambda x:porter_stemmer.stem(x))


In [9]:
docs_panda

,text
0,king man
1,king marries queen
2,queen marries king
3,unrelated works
4,queen woman


# Perform these operations if we have list of words !

In [10]:
#using a panda dataset
import pandas as pd 
docs_panda = pd.DataFrame()
docs_panda['text'] = docs_temp
docs_panda

,text
0,"[King, is, man]"
1,"[a, king, marries, queen]"
2,"[a, queen, marries, king]"
3,"[some, unrelated, works]"
4,"[queen, is, woman]"


In [11]:
docs_panda['text'].iloc[0].append(',')

In [12]:
# - making any word starts with lowercase 
docs_panda['text'] = docs_panda['text'].apply(lambda x: [word.lower() for word in x])
# - removing stopwords
stop = stopwords.words('english')
docs_panda['text'] = docs_panda['text'].apply(lambda x: [word for word in x if word not in stop])
#- removing punctuation 
docs_panda['text'] = docs_panda['text'].apply(lambda x: [word for word in x if word not in string.punctuation])

# performing stemming (#careful with this check)
# porter_stemmer = PorterStemmer()
# docs_panda['text'] = docs_panda['text'].apply(lambda x:[porter_stemmer.stem(word) for word in x])

In [13]:
docs_panda

,text
0,"[king, man]"
1,"[king, marries, queen]"
2,"[queen, marries, king]"
3,"[unrelated, works]"
4,"[queen, woman]"


# One Hot Encoding 

considering the dataset containing words as list 

In [14]:
#counting number of total words 
number_of_words = [(x) for x in docs_panda['text']]
len(set(chain(*number_of_words)))

7

In [15]:
vocab_size = 20
docs_panda['encoded'] = docs_panda['text'].apply(lambda x: [one_hot(word,vocab_size) for word in x])

In [16]:
docs_panda['encoded'] = docs_panda['encoded'].apply(lambda x: list(chain(*x)))


In [17]:
docs_panda

,text,encoded
0,"[king, man]","[12, 13]"
1,"[king, marries, queen]","[12, 2, 11]"
2,"[queen, marries, king]","[11, 2, 12]"
3,"[unrelated, works]","[3, 1]"
4,"[queen, woman]","[11, 16]"


In [18]:
#padding
# we are justing filling with 0 to make all sentences of the same length 
max_length = 10 
docs_panda['encoded'] = docs_panda['encoded'].apply(lambda x: pad_sequences([x], maxlen=max_length, padding='post'))


In [19]:
docs_panda

,text,encoded
0,"[king, man]","[[12, 13, 0, 0, 0, 0, 0, 0, 0, 0]]"
1,"[king, marries, queen]","[[12, 2, 11, 0, 0, 0, 0, 0, 0, 0]]"
2,"[queen, marries, king]","[[11, 2, 12, 0, 0, 0, 0, 0, 0, 0]]"
3,"[unrelated, works]","[[3, 1, 0, 0, 0, 0, 0, 0, 0, 0]]"
4,"[queen, woman]","[[11, 16, 0, 0, 0, 0, 0, 0, 0, 0]]"


In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('./wiki-news-300d-1M.vec')


In [ ]:
#king
#queen
#woman
#man
example = pd.DataFrame([[model['king']],[model['queen']],[model['woman']],[model['man']]])
df = example[0].apply(pd.Series)


In [ ]:
#apply pca to reduce and visualize
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pC = pca.fit_transform(df)
principalDf = pd.DataFrame(data = pC, columns = ['principal component 1', 'principal component 2'])

In [ ]:
pC

In [ ]:
pC[0]-pC[3]+pC[2]

# Text classification with word Embeddings

In [20]:
docs = ["Well done!",
        "Good work",
        "Great effort",
        "nice work",
        "excellent",
        "weak",
        "pour effort",
        "not good",
        "poor work",
        "could have done better"]
labels = [1,1,1,1,1,0,0,0,0,0]
        


In [21]:
#integer encode of document
from keras.preprocessing.text import one_hot
vocab_size = 50
encoded_docs = [one_hot(sentence,vocab_size) for sentence in docs]

In [22]:
encoded_docs

[[4, 39],
 [13, 35],
 [28, 4],
 [15, 35],
 [11],
 [34],
 [49, 4],
 [31, 13],
 [19, 35],
 [21, 22, 39, 40]]

In [23]:
# pad document to a max length of 4 words
max_length = 4 
padded_docs = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')
padded_docs

array([[ 4, 39,  0,  0],
       [13, 35,  0,  0],
       [28,  4,  0,  0],
       [15, 35,  0,  0],
       [11,  0,  0,  0],
       [34,  0,  0,  0],
       [49,  4,  0,  0],
       [31, 13,  0,  0],
       [19, 35,  0,  0],
       [21, 22, 39, 40]])

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Input


model = Sequential()
model.add(Embedding(vocab_size , 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
#compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
model.fit(padded_docs, labels, epochs=5, verbose=1)
#evaluate model
loss,accuracy = model.evaluate(padded_docs,labels, verbose=1)


Epoch 1/5
10/10 [==============================] - 2s 173ms/step - loss: 0.6953 - acc: 0.6000
Epoch 2/5
10/10 [==============================] - 0s 200us/step - loss: 0.6941 - acc: 0.5000
Epoch 3/5
10/10 [==============================] - 0s 100us/step - loss: 0.6929 - acc: 0.5000
Epoch 4/5
10/10 [==============================] - 0s 300us/step - loss: 0.6917 - acc: 0.6000
Epoch 5/5
10/10 [==============================] - 0s 4ms/step


# Provaaaaa

In [145]:
docs = ["La mamma ha fatto gli ! ! ! gnocchi",
        "Tonio Cartonio si fa i pippotti",
        'i pippotti li vende la mamma', 
        'la mamma si sbatte tonio cartonio',
        "Gli gnocchi producono pippotti"]

In [115]:
#lower 
docs = list(map(lambda sentence: sentence.lower(), docs))
docs

['la mamma ha fatto gli gnocchi',
 'tonio cartonio si fa i pippotti',
 'i pippotti li vende la mamma',
 'la mamma si sbatte tonio cartonio',
 'gli gnocchi producono pippotti']

In [148]:
docs = [elem.split() for elem in docs]


In [149]:
df = pd.DataFrame()
df['Text'] = docs

In [150]:
df

,Text
0,"[La, mamma, ha, fatto, gli, !, !, !, gnocchi]"
1,"[Tonio, Cartonio, si, fa, i, pippotti]"
2,"[i, pippotti, li, vende, la, mamma]"
3,"[la, mamma, si, sbatte, tonio, cartonio]"
4,"[Gli, gnocchi, producono, pippotti]"


In [151]:
df['Text'] = df['Text'].apply(lambda x: [word.lower() for word in x])

In [152]:
from nltk.corpus import stopwords

stop = stopwords.words('italian')


In [153]:
df['Text'] = df['Text'].apply(lambda x: [word for word in x if word not in stop])

In [154]:
df

,Text
0,"[mamma, fatto, !, !, !, gnocchi]"
1,"[tonio, cartonio, fa, pippotti]"
2,"[pippotti, vende, mamma]"
3,"[mamma, sbatte, tonio, cartonio]"
4,"[gnocchi, producono, pippotti]"


In [144]:
from string import punctuation as punct

In [156]:
df['Text'] = df['Text'].apply(lambda x : [word for word in x if word not in punct])

In [157]:
#Embedding words 

In [191]:
from keras.preprocessing.text import one_hot
from itertools import chain

In [192]:
vocab_size = 30
df['Encoding'] = df['Text'].apply(lambda x :[one_hot(word,vocab_size) for word in x] )

In [193]:
df['Encoding'] = df['Encoding'].apply(lambda x : list(chain(*x)))

In [194]:
df

,Text,Encoding
0,"[mamma, fatto, gnocchi]","[3, 10, 4]"
1,"[tonio, cartonio, fa, pippotti]","[17, 15, 26, 13]"
2,"[pippotti, vende, mamma]","[13, 21, 3]"
3,"[mamma, sbatte, tonio, cartonio]","[3, 25, 17, 15]"
4,"[gnocchi, producono, pippotti]","[4, 21, 13]"


In [195]:
from keras.preprocessing.sequence import pad_sequences

In [196]:
max_length = df['Encoding'].map(len).max()
vocab_size 
df['Encoding'] = df['Encoding'].apply(lambda x: pad_sequences([x],maxlen=max_length, padding='post'))

In [197]:
df['Encoding']=list(map(lambda x : x.flatten(),df['Encoding']))

In [198]:
df

,Text,Encoding
0,"[mamma, fatto, gnocchi]","[3, 10, 4, 0]"
1,"[tonio, cartonio, fa, pippotti]","[17, 15, 26, 13]"
2,"[pippotti, vende, mamma]","[13, 21, 3, 0]"
3,"[mamma, sbatte, tonio, cartonio]","[3, 25, 17, 15]"
4,"[gnocchi, producono, pippotti]","[4, 21, 13, 0]"


In [ ]:
input